In [2]:

import os
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
import numpy as np
from rasterio.transform import from_origin
from rasterio.windows import Window



### Cargamos la imagen con rasterio

Rasterio es la liberería que permitirá que las imágenes conserven los metadatos y que estén georeferenciadas aún después de procesarlas y recortarlas. 

In [32]:

# path  = '/content/drive/My Drive/doctorado_albert/conteo_pinguinos/'
# image_name = "orthomosaic_all_big.tif"

NUM_TILE = 81

if os.path.exists('/content/drive/My Drive/doctorado_albert/conteo_pinguinos/'):
    path  = '/content/drive/My Drive/doctorado_albert/conteo_pinguinos/'
else:
    path = 'G:\\.shortcut-targets-by-id\\1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG\\doctorado_albert\\conteo_pinguinos'

image_name = f"recortes/recorte_{NUM_TILE}.tif"
tiff_file = os.path.join(path, image_name)


#! Verificamos la carga de la imagen (Solamente obtenemos información)
with rasterio.open(tiff_file) as src:
    print(f"Imagen cargada desde: {tiff_file}")
    print("Dimensiones: ", (src.width, src.height))
    print("Transformación de coordenadas: ", src.transform)
    print("Sistema de coordenadas: ", src.crs)

Imagen cargada desde: G:\.shortcut-targets-by-id\1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG\doctorado_albert\conteo_pinguinos\recortes/recorte_81.tif
Dimensiones:  (10195, 11420)
Transformación de coordenadas:  | 0.00, 0.00,-59.24|
| 0.00,-0.00,-62.31|
| 0.00, 0.00, 1.00|
Sistema de coordenadas:  EPSG:4326


### Obtención de las coordenadas y de variables necesarias de la imagen

In [33]:
with rasterio.open(tiff_file) as src:
    # Obtener los metadatos
    transform = src.transform
    metadata = src.meta
    top_left = src.transform * (0, 0)
    top_right = src.transform * (src.width, 0)
    bottom_left = src.transform * (0, src.height)
    bottom_right = src.transform * (src.width, src.height)

    WIDTH, HEIGHT = src.width, src.height

   
    print("Metadatos de la imagen:")
    for key, value in metadata.items():
        print(f"{key}: {value}")

    # También puedes acceder a información específica si lo deseas
    print("\nInformación adicional:")
    print("Ancho de la imagen:", src.width)
    print("Altura de la imagen:", src.height)
    print("Sistema de coordenadas:", src.crs)  # Sistema de referencia de coordenadas
    print("Transformación:", src.transform)  # Transformación espacial


    print("Coordenadas de las esquinas de la imagen:")
    print("TOP LEFT:", top_left)
    print("Esquina superior derecha:", top_right)
    print("Esquina inferior izquierda:", bottom_left)
    print("BOTTOM RIGHT:", bottom_right)

  

Metadatos de la imagen:
driver: GTiff
dtype: uint8
nodata: None
width: 10195
height: 11420
count: 4
crs: EPSG:4326
transform: | 0.00, 0.00,-59.24|
| 0.00,-0.00,-62.31|
| 0.00, 0.00, 1.00|

Información adicional:
Ancho de la imagen: 10195
Altura de la imagen: 11420
Sistema de coordenadas: EPSG:4326
Transformación: | 0.00, 0.00,-59.24|
| 0.00,-0.00,-62.31|
| 0.00, 0.00, 1.00|
Coordenadas de las esquinas de la imagen:
TOP LEFT: (-59.241644524786786, -62.30645467979474)
Esquina superior derecha: (-59.236389267356785, -62.30645467979474)
Esquina inferior izquierda: (-59.241644524786786, -62.30919488595474)
BOTTOM RIGHT: (-59.236389267356785, -62.30919488595474)


## Definición de la carpeta de salida

La carpeta de salida para las imágenes recortadas se llama 'cut_files', y aquí solamente estarán los tiles recortados en una aproximación de 500x500 (en realidad será un poco más, ya que el recorte de 20 filas y 20 columnas no es exacto).

In [34]:
#Directorio de salida
OUTPUT_DIR = os.path.join('cut_tiles', f'tiles_500x500_{NUM_TILE}')
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
print(f"Output directory: {OUTPUT_DIR}")

Output directory: cut_tiles\tiles_500x500_81


## Recorte de los tiles

1. Se define en cuánto se quiere recortar (en nuestro caso 20 filas y 20 columnas). 
2. Se descartan todos los tiles que tengan una alta proporción de píxeles blanco o negros (mnás del 50%)
3. Acto seguido se procede al recorte y se envía a la carpeta de OUPUT_DIR.

In [35]:
import numpy as np
from rasterio.windows import Window

# Definición del recorte
ROWS, COLS = 20, 20

# Dimensiones de cada recorte
tile_width = WIDTH // COLS
tile_height = HEIGHT // ROWS

# Crear un directorio para las imágenes recortadas
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Abrir el TIFF original con rasterio
with rasterio.open(tiff_file) as src:
    for i in range(ROWS):
        print("Fila nº ", i)
        for j in range(COLS):
            left = j * tile_width
            upper = i * tile_height
            
            # Definir ventana de recorte
            window = Window(left, upper, tile_width, tile_height)
            
            # Leer el recorte
            cropped_image = src.read(window=window)

            # Identificar píxeles negros (000000)
            is_black = np.all(cropped_image == 0, axis=0)
            
            # Identificar píxeles blancos (FFFFFF)
            is_white = np.all(cropped_image == 255, axis=0)
            
            # Identificar píxeles vacíos o transparentes (suponiendo valor 0 en el canal de alfa o nodata)
            if "nodata" in src.profile:
                nodata_value = src.profile["nodata"]
                is_empty = np.any(cropped_image == nodata_value, axis=0)
            else:
                # Si hay un canal alfa, considera valores 0 como transparentes
                is_empty = cropped_image.shape[0] > 3 and np.all(cropped_image[-1] == 0, axis=0)

            # Calcular proporciones de píxeles negros, blancos y vacíos
            num_pixels = cropped_image.shape[1] * cropped_image.shape[2]  # Total de píxeles
            black_ratio = np.sum(is_black) / num_pixels
            white_ratio = np.sum(is_white) / num_pixels
            empty_ratio = np.sum(is_empty) / num_pixels

            # Guardar solo si todos los umbrales se cumplen
            if black_ratio <= 0.5 and white_ratio <= 0.1 and empty_ratio <= 0.1:
                # Crear los metadatos del recorte
                cropped_meta = src.meta.copy()
                cropped_meta.update({
                    "height": tile_height,
                    "width": tile_width,
                    "transform": rasterio.windows.transform(window, src.transform)
                })
                
                # Guardar la imagen recortada con georreferenciación
                output_path = f"{OUTPUT_DIR}/tile_{NUM_TILE}_subrecorte_{i * COLS + j + 1}.tiff"
                with rasterio.open(output_path, 'w', **cropped_meta) as dst:
                    dst.write(cropped_image)
                print(f"Guardado recorte {i * COLS + j + 1} en {output_path}")
            else:
                print(f"Recorte {i * COLS + j + 1} descartado por exceso de píxeles negros, blancos o vacíos")

    


Fila nº  0
Guardado recorte 1 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_1.tiff
Guardado recorte 2 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_2.tiff
Guardado recorte 3 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_3.tiff
Guardado recorte 4 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_4.tiff
Guardado recorte 5 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_5.tiff
Guardado recorte 6 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_6.tiff
Guardado recorte 7 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_7.tiff
Guardado recorte 8 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_8.tiff
Guardado recorte 9 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_9.tiff
Guardado recorte 10 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_10.tiff
Guardado recorte 11 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_11.tiff
Guardado recorte 12 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_12.tiff
Guardado recorte 13 en cut_tiles\tiles_500x500_81/tile_81_subrecorte_13.tiff
Guarda

### Comprobamos que los recortes están efectivamente georeferenciados

El código que hay a continuación solamente verifica que la georeferencia es correcta

In [18]:
TEST_TILE = os.path.join(f'tile_{NUM_TILE}_subrecorte_{NUM_TILE}.tiff')
sample_file = os.path.join(OUTPUT_DIR, TEST_TILE)

with rasterio.open(sample_file) as src:
    transform = src.transform
    metadata = src.meta
    top_left = src.transform * (0, 0)
    top_right = src.transform * (src.width, 0)
    bottom_left = src.transform * (0, src.height)
    bottom_right = src.transform * (src.width, src.height)

    WIDTH, HEIGHT = src.width, src.height

   
    print(f"Metadatos de la imagen {sample_file}:")
    for key, value in metadata.items():
        print(f"{key}: {value}")

    # También puedes acceder a información específica si lo deseas
    print("\nInformación adicional:")
    print("Ancho de la imagen:", src.width)
    print("Altura de la imagen:", src.height)
    print("Sistema de coordenadas:", src.crs)  # Sistema de referencia de coordenadas
    print("Transformación:", src.transform)  # Transformación espacial


    print("Coordenadas de las esquinas de la imagen:")
    print("TOP LEFT:", top_left)
    print("Esquina superior derecha:", top_right)
    print("Esquina inferior izquierda:", bottom_left)
    print("BOTTOM RIGHT:", bottom_right)

RasterioIOError: cut_tiles\tiles_500x500_61\tile_61_subrecorte_61.tiff: No such file or directory